In [49]:
# 필요한 라이브러리 설정
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.testing import make_tensor
from pytorch_model_summary import summary
from torch.utils.tensorboard import SummaryWriter

from os.path import join
from tqdm import tqdm
import datetime

from CBDtorch.vaelstm import *
from CBDtorch.custom import Dataset4autoencoder
from CBDtorch.dirs import *


In [43]:
# 일단 확인용으로 VAELSTM 만들기
dataType = 'angle'
numFold = 0
seq_len = 101
num_features = 42
embedding_dim = 30 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
my_model = VariationalEncoder(seq_len, num_features, embedding_dim, device)
filename = f'{dataType}_{embedding_dim}_{numFold}_fold'
filename = join('.',filename)
torch.save(my_model.state_dict(), join('.',filename)) 

In [55]:
# 메인 아이디어
class regressor(nn.Module):
    def __init__(self,filename,emb_dims,*args):
        super().__init__()
        self.VAE = VariationalEncoder(*args)
        self.VAE.load_state_dict(torch.load(filename))
        self.dense = nn.Sequential(nn.Linear(emb_dims,2048),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.5),  # 노드를 학습과정에서 얼만큼 활용 안할지
                                        nn.Linear(2048,2048),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.5),
                                        nn.Linear(2048,1024),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.5),
                                        nn.Linear(1024,512),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.2),
                                        nn.Linear(512,512),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.2),
                                        nn.Linear(512,303),
                                        )
        ##You can use as many linear layers and other activations as you want
    def forward(self, x):
        x = self.VAE(x)
        output = self.dense(x)
        return output

In [56]:
# 모델 불러오기
reg = regressor(filename, embedding_dim, seq_len, num_features, embedding_dim, device)

In [57]:
# show input shape
print(summary(reg, torch.zeros(32, 101, 42), show_input=True))
# show output shape
print(summary(reg, torch.zeros(32, 101, 42), show_input=False))

----------------------------------------------------------------------------
           Layer (type)         Input Shape         Param #     Tr. Param #
   VariationalEncoder-1       [32, 101, 42]          90,060          90,060
               Linear-2            [32, 30]          63,488          63,488
                 ReLU-3          [32, 2048]               0               0
              Dropout-4          [32, 2048]               0               0
               Linear-5          [32, 2048]       4,196,352       4,196,352
                 ReLU-6          [32, 2048]               0               0
              Dropout-7          [32, 2048]               0               0
               Linear-8          [32, 2048]       2,098,176       2,098,176
                 ReLU-9          [32, 1024]               0               0
             Dropout-10          [32, 1024]               0               0
              Linear-11          [32, 1024]         524,800         524,800
           

In [58]:
nn.L1Loss()(10,10)

AttributeError: 'int' object has no attribute 'size'